# 1. 데이터 전처리

In [2]:
import pandas as pd

In [26]:
starbucks = pd.read_csv('/content/drive/MyDrive/KDT/9_데이터분석/data/starbucks_seoul.csv')
starbucks = starbucks.set_axis(['지점명', '지점주소', '지점위도', '지점경도'], axis=1).reset_index(drop=True)

In [27]:
starbucks.head()

,지점명,지점주소,지점위도,지점경도
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [9]:
store = pd.read_csv("/content/drive/MyDrive/KDT/9_데이터분석/data/소상공인시장진흥공단_상가(상권)정보_서울_202510.csv", low_memory=False)
store.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [11]:
store_column = ["이디야", "이디야커피", "올리브영", "메가커피", "메가엠지씨", '메가MGC', "컴포즈커피", "백다방", "빽다방"]

In [ ]:
# case=False : 대소문자를 구분하지 않음
# na=False : 결측치가 있는 경우 False로 간주
store[store["상호명"].str.contains("메가MGC", case=False, na=False)]

In [15]:
store_copy = store.copy()

In [21]:
# 특정 문자열을 포함하고 있으면 그 문자열을 반환
store_copy["상호명"] = store_copy["상호명"].str.extract("({})".format("|".join(store_column)))
store_copy["상호명"] = store_copy["상호명"].dropna()

In [23]:
store_copy = store_copy.dropna(subset=['상호명']).iloc[:, [0, 1, 14, 37, 38]].reset_index(drop=True)
store_copy.head()

,상가업소번호,상호명,시군구명,경도,위도
0,MA010120220801142601,이디야,영등포구,126.888601,37.526796
1,MA010120220808836788,메가커피,서초구,127.002409,37.503936
2,MA010120220812797025,컴포즈커피,종로구,126.987638,37.578016
3,MA010120220814215158,컴포즈커피,도봉구,127.027834,37.660084
4,MA010120220808574817,빽다방,광진구,127.093411,37.535353


In [24]:
store_copy["상호명"] = store_copy["상호명"].str.replace("메가엠지씨", "메가커피", regex=False)
store_copy["상호명"] = store_copy["상호명"].str.replace("메가MGC", "메가커피", regex=False)
store_copy["상호명"] = store_copy["상호명"].str.replace("이디야커피", "이디야", regex=False)
store_copy["상호명"] = store_copy["상호명"].str.replace("백다방", "빽다방", regex=False)

In [25]:
print(store_copy.shape)
print(starbucks.shape)

(2539, 5)
(666, 4)


In [29]:
df_cross = store_copy.merge(starbucks, how="cross")
df_cross.head()

,상가업소번호,상호명,시군구명,경도,위도,지점명,지점주소,지점위도,지점경도
0,MA010120220801142601,이디야,영등포구,126.888601,37.526796,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,MA010120220801142601,이디야,영등포구,126.888601,37.526796,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,MA010120220801142601,이디야,영등포구,126.888601,37.526796,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,MA010120220801142601,이디야,영등포구,126.888601,37.526796,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,MA010120220801142601,이디야,영등포구,126.888601,37.526796,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


# 2. 하버사인 공식

Haversine 공식은 구체(구면) 위의 두 점(위도와 경도로 표시됨) 사이의 최단 거리(대원 거리, great-circle distance)를 계산하는 방법입니다. 이 공식은 GPS 좌표(위도와 경도)를 활용하여 지구 표면 위의 거리 계산에 널리 사용됩니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FSrWGW%2FbtsLpNU6kSG%2FAAAAAAAAAAAAAAAAAAAAACWyPfRAMbpe0b_v6ut8rDPwdQDwSdWFmpo2ovU3Wj0f%2Fimg.png%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1767193199%26allow_ip%3D%26allow_referer%3D%26signature%3DhRR5Fl5OWcNdVJE7fsXfXDXvJ%252BY%253D">

In [31]:
import pandas as pd
from haversine import haversine

In [32]:
df_cross["거리"] = df_cross.apply(lambda x:haversine([x["위도"], x["경도"]], [x["지점위도"], x["지점경도"]], unit="m"), axis=1)

In [38]:
# 개별 매장들과 스타벅스와의 최소 거리
df_dis = df_cross.groupby(["상가업소번호", "상호명"])["거리"].min().reset_index()
df_dis.head()

,상가업소번호,상호명,거리
0,MA010120220800021713,컴포즈커피,692.058445
1,MA010120220800024620,이디야,95.740366
2,MA010120220800026176,이디야,1115.073437
3,MA010120220800033480,이디야,81.912918
4,MA010120220800033806,이디야,351.880382


In [39]:
# 각 프랜차이즈 별 스타벅스와의 평균 거리
df_dis.groupby("상호명")["거리"].mean()

,거리
상호명,
메가커피,376.455889
빽다방,299.957329
올리브영,200.563616
이디야,378.976138
컴포즈커피,344.453309


In [40]:
# agg(): 다중 집계 작업을 간단하게 해주는 함수
df_dis.groupby(["상호명"])["거리"].agg(["mean", "count"])

,mean,count
상호명,,
메가커피,376.455889,861
빽다방,299.957329,360
올리브영,200.563616,432
이디야,378.976138,450
컴포즈커피,344.453309,436


In [41]:
def distance(x):
  dis = df_dis["거리"] <= x
  return df_dis[dis].groupby("상호명")["거리"].agg(["mean", "count"])

In [42]:
distance(100)

,mean,count
상호명,,
메가커피,60.831928,152
빽다방,63.264894,90
올리브영,52.499012,197
이디야,67.858056,81
컴포즈커피,62.479946,88


# 3. 차트 시각화

In [46]:
import IPython
from pandasecharts import echart

In [45]:
df_50 = distance(50).reset_index()
df_50.head()

,상호명,mean,count
0,메가커피,29.196363,47
1,빽다방,35.566886,28
2,올리브영,23.499137,82
3,이디야,34.408563,19
4,컴포즈커피,31.876068,28


In [48]:
df_50.echart.pie(x='상호명', y='count', figsize=(600, 400),
                  radius=['20%', '60%'], label_opts={'position':'outer'},
                  title='커피 프렌차이즈의 입점전략은 과연 스타벅스 옆인가?',
                  legend_opts={'pos_right':'0%', 'orient':'vertical'},
                  subtitle='50m 이내 매장수', init_opts={'bg_color': 'white'}).render()

IPython.display.HTML(filename='render.html')

In [49]:
from pyecharts.charts import Timeline, Grid

tl = Timeline({'width':'600px', 'height':'400px'})

for i in [1000, 100, 50, 30]:
    df_d = distance(i).reset_index()
    pie1 = df_d.echart.pie(x='상호명', y='count', figsize=(600, 400),
                    radius=['20%', '60%'], label_opts={'position':'outer'},
                    title='커피 프렌차이즈의 입점전략은 과연 스타벅스 옆인가?',
                    legend_opts={'pos_right':'0%', 'orient':'vertical'},
                    subtitle='{}m 이내 매장수'.format(i), init_opts={'bg_color': 'white'})
    tl.add(pie1, '{}m'.format(i)).render()

IPython.display.HTML(filename='render.html')